Methods of Machine Learning and Intellectual Analysis of Data (Learning course) \
Bohdan Pavlyshenko (https://www.linkedin.com/in/bpavlyshenko/) \
Youtube video: https://www.youtube.com/watch?v=muHojPnCcGU&list=PLMQt7tnruMvF1jetHNUKjMtHKSp78H2bb&index=15

# Text Regression (Product Price Prediction Using Text Description)

Used Resource: \
https://www.kaggle.com/c/mercari-price-suggestion-challenge    

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold, train_test_split
import math
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from scipy.sparse import csr_matrix, hstack
import xgboost as xgb

In [2]:
train = pd.read_table("data/mercary_price_train.tsv")

In [3]:
train.head(3)

train_id                                 name  item_condition_id  \
0         0  MLB Cincinnati Reds T Shirt Size XL                  3   
1         1     Razer BlackWidow Chroma Keyboard                  3   
2         2                       AVA-VIV Blouse                  1   

                                       category_name brand_name  price  \
0                                  Men/Tops/T-shirts        NaN   10.0   
1  Electronics/Computers & Tablets/Components & P...      Razer   52.0   
2                        Women/Tops & Blouses/Blouse     Target   10.0   

   shipping                                   item_description  
0         1                                 No description yet  
1         0  This keyboard is in great condition and works ...  
2         1  Adorable top with a hint of lace and a key hol...

In [6]:
train.brand_name.nunique(), train.category_name.nunique(), train.shape

(4809, 1287, (1482535, 8))

In [7]:
def fill_na(df):
    df.category_name.fillna(value="na", inplace=True)
    df.brand_name.fillna(value="na", inplace=True)
    df.item_description.fillna(value="na", inplace=True)
    return (df)

train =fill_na(train)

le = LabelEncoder()
le.fit(train.category_name)
train.category_name = le.transform(train.category_name)

le.fit(train.brand_name)
train.brand_name = le.transform(train.brand_name)

train.head(3)

train_id                                 name  item_condition_id  \
0         0  MLB Cincinnati Reds T Shirt Size XL                  3   
1         1     Razer BlackWidow Chroma Keyboard                  3   
2         2                       AVA-VIV Blouse                  1   

   category_name  brand_name  price  shipping  \
0            808        4788   10.0         1   
1             86        3557   52.0         0   
2           1254        4180   10.0         1   

                                    item_description  
0                                 No description yet  
1  This keyboard is in great condition and works ...  
2  Adorable top with a hint of lace and a key hol...

In [8]:
tfidf = TfidfVectorizer(max_features=30000,ngram_range=(1, 3),stop_words='english')
tfidf_v = tfidf.fit_transform(train['item_description']+' '+train['name'])

In [9]:
lb = LabelBinarizer(sparse_output=True)
brand_b = lb.fit_transform(train['brand_name'])

In [10]:
lb = LabelBinarizer(sparse_output=True)
categ_b = lb.fit_transform(train['category_name'])

In [11]:
dumm_v = csr_matrix(pd.get_dummies(train[['item_condition_id', 'shipping']],sparse=True).values)

In [12]:
x_sparse=hstack([tfidf_v,categ_b,brand_b,dumm_v]).tocsr()

In [13]:
y_train = np.log1p(train["price"])

In [14]:
dtrain, dvalid, y_train, y_val = train_test_split(x_sparse, y_train, random_state=1, test_size=0.15)

In [15]:
d_train = xgb.DMatrix(dtrain, label=y_train)
d_val = xgb.DMatrix(dvalid, label=y_val)

In [17]:
params = {}
params['objective'] = 'reg:linear'
params['eta'] = 0.1
params['max_depth'] = 15
params['silent'] = 1
params['subsample']=1
params['colsample_bytree']=0.75
params['gamma']=0
params['min_child_weight']=1
params['scale_pos_weight']=1
params['eval_metric'] = 'rmse'
watchlist = [(d_train, 'train'), (d_val, 'val')]
clf = xgb.train(params, d_train, 500, watchlist, early_stopping_rounds=5, verbose_eval=10)



[10:13:04] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[10:13:04] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:2.34798	val-rmse:2.34950
[10]	train-rmse:0.99747	val-rmse:1.00058
[20]	train-rmse:0.64454	val-rmse:0.65234
[30]	train-rmse:0.57039	val-rmse:0.58210
[40]	train-rmse:0.55035	val-rmse:0.56421
[50]	train-rmse:0.53945	val-rmse:0.55513
[60]	train-rmse:0.53035	val-rmse:0.54785
[70]	train-rmse:0.52286	val-rmse:0.54191
[80]	tr

In [25]:
pred = clf.predict(d_val)

In [26]:
rmsle_v = np.sqrt(np.mean((pd.Series(pred.tolist())-pd.Series(y_val.tolist()))**2))
print(f" RMSLE : {rmsle_v}")

 RMSLE : 0.47758048322060737
